In [7]:
import pandas as pd

# Cargar ambos datasets
df = pd.read_csv('datasets/combined_dataset.csv')  # Reemplaza con la ruta de tu primer archivo

label_counts = df['label'].value_counts()

# Mostrar los resultados
print("Conteo de etiquetas:")
print(label_counts)

nulos_por_columna = df.isnull().sum()

# Mostrar solo las columnas que tienen valores nulos
nulos_con_valores = nulos_por_columna[nulos_por_columna > 0]

if nulos_con_valores.empty:
    print("No hay valores nulos en ninguna columna.")
else:
    print("Columnas con valores nulos:")
    print(nulos_con_valores)
df = df.dropna(subset=['text'])


df.head()


Conteo de etiquetas:
label
Ham     102159
Spam     91739
Name: count, dtype: int64
No hay valores nulos en ninguna columna.


,label,text
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...
1,Ham,got ice thought look az original message ice o...
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...
3,Spam,start increasing your odds of success & live s...
4,Ham,author jra date escapenumber escapenumber esca...


## Cargar y Preprocesar los Datos
Cargamos los datos y realizamos una codificación de etiquetas. Además, dividimos el dataset en entrenamiento + validación y prueba, manteniendo la proporción de clases en cada conjunto.


In [9]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Los pasamos a lista ya que los tokenizadores necesitan listas y no series 
texts = df["text"].tolist()
labels = df['label'].tolist()

# Codificar etiquetas (Spam = 1, Ham = 0)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Dividir datos en entrenamiento y validación
# Hacemos una division estartificada para que todos los conjuntos tengan la misma proporcion de spam o ham

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, stratify=labels, random_state=42)

# Variables obtenidas aquí:
# - test_texts: lista de textos para el conjunto de prueba (20% del dataset original).
# - test_labels: lista de etiquetas correspondientes para el conjunto de prueba.

# Segundo split: separar el conjunto de entrenamiento+validación en entrenamiento y validación

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.25, stratify=train_labels, random_state=42)

# Variables obtenidas aquí:
# - train_texts: lista de textos para el conjunto de entrenamiento final (60% del dataset original).
# - train_labels: lista de etiquetas correspondientes para el conjunto de entrenamiento final.
# - val_texts: lista de textos para el conjunto de validación (20% del dataset original).
# - val_labels: lista de etiquetas correspondientes para el conjunto de validación.


## Tokenización de Texto
Convertimos el texto en una secuencia de números usando el tokenizador de BERT. Esto facilita que BERT procese el texto

In [10]:
# Cargar el tokenizador preentrenado de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(sentences):
    return tokenizer(
        sentences,
        max_length=128,           # Limita la longitud de cada entrada
        padding='max_length',     # Rellena las oraciones más cortas
        truncation=True,          # Recorta oraciones largas
        return_tensors='tf'       # Retorna tensores compatibles con TensorFlow
    )

# Tokenizar cada conjunto de datos
train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
test_encodings = tokenize(test_texts)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

2024-11-12 11:44:25.838361: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Construcción del Modelo Usando BERT
Usamos BERT como base del modelo, agregando una capa de salida para clasificar los correos como spam o ham.

In [11]:
# Cargar el modelo BERT preentrenado
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Definir las entradas del modelo , en la que todo lo que entra se tokeniza y se indican los token con mas o menos atencion
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Pasamos estas entradas a bert

embeddings = bert_model(input_ids, attention_mask=attention_mask)[1] 

# Añadimos una capa de salida con activación sigmoide que es ideal para tareas de clasificación binaria porque convierte la salida en una probabilidad
# combinamos las entradas y la salida
output = tf.keras.layers.Dense(1, activation="sigmoid")(embeddings)

# Definir el modelo completo
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# Entrenar el Modelo

In [12]:
history = model.fit(
    x={'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    y=train_labels,
    validation_data=(
        {'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask']},
        val_labels
    ),
    batch_size=16,
    epochs=3
)


Epoch 1/3
  23/7272 [..............................] - ETA: 5:14:42 - loss: 0.5613 - accuracy: 0.7174

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(
    x={'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    y=test_labels
)

print(f"Accuracy en prueba: {test_accuracy:.2f}")
